In [2]:
from IPython.display import display, HTML

display(
    HTML(
        """<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""
    )
)

# 문장 - 벡터 (1차원 숫자 배열 [8.1, 9.1, 2, 5, 4, 3])

- openai API : https://platform.openai.com/ 의 키(OPENAI_API_KEY)를 .env 등록
- upstage : https://console.upstage.ai/ 의 키(UPSTAGE_API_KEY)를 .env 등록


# 1. 환경변수 load

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 2. 유사도 게싼하는 방법 : https://www.pinecone.io/learn/vector-similarity 

1. 유클리드 거리 : 두 벡터간의 거리가 가까운지
2. 코사인유사도: 두 벡터간 방향이 유사한지
3. dot product : 두 벡터간의 곱을 사용하여 거리와 방향을 모두 고려

In [14]:
import numpy as np
def cosine_similarity(vec1, vec2):
    """두 백터 사이의 코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1) # 벡터의 길이
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1==0 or norm_vec2==0:
        return 0.0
    return dot_product / (norm_vec1*norm_vec2)

# 3. openAI API 의 embedding model 사용 
- 한글의 경우 유사도 측정이 별루 좋지 않다. 그래서 upstage 를 이용

In [ ]:
from openai import OpenAI
openai_client = OpenAI()

In [ ]:
# text-embedding-3-large
# 영어 문장을 벡터로 바꿔도 유사도는 비슷해야 할 듯
response = openai_client.embeddings.create(
    input="king",
    model="text-embedding-3-large"
)

In [9]:
import numpy as np
king_vector = np.array(response.data[0].embedding)
print(king_vector.shape)
print(king_vector)

(3072,)
[ 0.01040417  0.02499519 -0.0014776  ...  0.00835009  0.01049861
 -0.00254005]


In [10]:
queen_response = openai_client.embeddings.create(
    input="queen",
    model="text-embedding-3-large"
)

In [12]:
queen_vector = np.array(queen_response.data[0].embedding)
print(queen_vector.shape)
print(queen_vector)

(3072,)
[-0.01385735  0.0008602  -0.0167823  ...  0.00017693  0.01159847
  0.00638929]


In [19]:
king_queen_similarity = cosine_similarity(king_vector, queen_vector)
print("king과 queen의 유사도 : ", king_queen_similarity)

king과 queen의 유사도 :  0.5552268369726675


In [16]:
slave_response = openai_client.embeddings.create(
    input="slave",
    model="text-embedding-3-large"
)

In [17]:
slave_vector = np.array(slave_response.data[0].embedding)
print(slave_vector.shape)
print(slave_vector)

(3072,)
[-0.02000881  0.00614965  0.01193179 ...  0.00095144 -0.02677098
 -0.00585972]


In [20]:
king_slave_similarity = cosine_similarity(king_vector, slave_vector)
print("king과 slave의 유사도 : ", king_slave_similarity)

king과 slave의 유사도 :  0.2948359989366531


In [ ]:
# 한국어 문장을 벡터로 바꿔도 유사도는 비슷해야 할 듯

In [23]:
kor_king_response = openai_client.embeddings.create(
    input="왕",
    model="text-embedding-3-large"
)

In [24]:
kor_king_vector = np.array(kor_king_response.data[0].embedding)
print(kor_king_vector.shape)
print(kor_king_vector)

(3072,)
[-0.00595223  0.01159333 -0.01316932 ... -0.00357134  0.01323696
 -0.00083999]


In [26]:
kor_queen_response = openai_client.embeddings.create(
    input="여왕",
    model="text-embedding-3-large"
)

In [27]:
kor_queen_vector = np.array(kor_queen_response.data[0].embedding)
print(kor_queen_vector.shape)
print(kor_queen_vector)

(3072,)
[-0.01307151 -0.00921458 -0.00532257 ... -0.00482468 -0.00204418
  0.02035061]


In [28]:
# 왕과 여왕의 유사
kor_king_queen_similarity = cosine_similarity(kor_king_vector, kor_queen_vector)
print("king과 slave의 유사도 : ", kor_king_queen_similarity)

king과 slave의 유사도 :  0.48733449549538954


In [29]:
kor_slave_response = openai_client.embeddings.create(
    input="거자",
    model="text-embedding-3-large"
)

In [30]:
# 왕과 거지의 유사
kor_slave_vector = np.array(kor_slave_response.data[0].embedding)
print(kor_slave_vector.shape)
print(kor_slave_vector)

(3072,)
[-0.00554396 -0.0408888  -0.00825767 ... -0.00328476 -0.01956536
  0.0113543 ]


In [31]:
kor_king_slave_similarity = cosine_similarity(kor_king_vector, kor_slave_vector)
print("king과 slave의 유사도 : ", kor_king_slave_similarity)

king과 slave의 유사도 :  0.21912431861711115


In [33]:
# king 과 왕의 유사도
king_korenKing_similarity = cosine_similarity(king_vector, kor_king_vector)
print("king과 slave의 유사도 : ", king_korenKing_similarity)

king과 slave의 유사도 :  0.5474873912140233


# 4. upstage의 embedding model 사용
- 한국에 embedding 에는 openai 의 embedding 데이터를 사용하는 것이 좋음

In [35]:
import os

upstage_api_key = os.environ.get("UPSTAGE_API_KEY")

upstage_client = OpenAI(
    api_key=upstage_api_key,
    base_url="https://api.upstage.ai/v1"
)

In [36]:
up_king_response = upstage_client.embeddings.create(
    input="king",
    model="embedding-query"
)

In [40]:
up_king_vector = np.array(up_king_response.data[0].embedding)
print(up_king_vector.shape)
print(up_king_vector)

(4096,)
[-0.01187134 -0.02062988 -0.00674057 ... -0.01081848  0.00247955
  0.01520538]


In [41]:
up_queen_response = upstage_client.embeddings.create(
    input="queen",
    model="embedding-query"
)

In [42]:
up_queen_vector = np.array(up_queen_response.data[0].embedding)
print(up_queen_vector.shape)
print(up_queen_vector)

(4096,)
[-0.00164413 -0.00951385 -0.00470734 ...  0.00989532 -0.00730133
  0.02600098]


In [43]:
up_king_queen_similarity = cosine_similarity(up_king_vector, up_queen_vector)
print("king과 slave의 유사도 : ", up_king_queen_similarity)

king과 slave의 유사도 :  0.6279103035110143


In [44]:
up_kor_king_response = upstage_client.embeddings.create(
    input="왕",
    model="embedding-query"
)

In [45]:
up_kor_king_vector = np.array(up_kor_king_response.data[0].embedding)
print(up_kor_king_vector.shape)
print(up_kor_king_vector)

(4096,)
[-0.01207733 -0.0224762  -0.01322937 ... -0.00020826  0.00362587
  0.01420593]


In [50]:
# upstage 에서의 king 과 왕의 유사도
up_king_korenKing_similarity = cosine_similarity(up_king_vector, up_kor_king_vector)
print(up_king_korenKing_similarity)

0.852188301320326
